<a href="https://colab.research.google.com/github/zzunebye/Capstone-code-data/blob/main/_RumorEval_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

/content
Mounted at ./MyDrive


In [7]:
cd MyDrive/MyDrive/Capstone/code_data

[Errno 2] No such file or directory: 'MyDrive/MyDrive/Capstone/code_data'
/content/MyDrive/MyDrive/Capstone/code_data


In [1]:
import pandas as pd
import numpy as np

from glob2 import glob
import json

In [35]:
import random


class data_loader():
    def __init__(self, path):
        self.path = path

    def getdata(self, source=True, split=True):
        jsons = glob('%s/*/*/*/source-tweet/*.json' % self.path)
        print("The number of json files are:", len(jsons))
        return [jsons]

    def extract_data(self, data):
        count = 0  # help var
        data_lists = []
        isRumorLists = []
        for index, dataset in enumerate(data):
            data_list = []
            isRumorList = []
            count = 0  # help var

            for jsonFile in dataset:
                count += 1
                if jsonFile.find("non-rumours") == -1:
                    isRumorList.append(1)
                else:
                    isRumorList.append(0)

                with open(jsonFile, 'r') as f:
                    for l in f.readlines():
                        if not l.strip():  # skip empty lines
                            continue
                        json_data = json.loads(l)
                        # print (json_data,"\n\n")
                        data_list.append(json_data)

            isRumorLists.append(pd.DataFrame(isRumorList, columns=['isRumor']))
            data_lists.append(data_list)
        return data_lists, isRumorLists


In [36]:
def tweets2tokens(tweet_text):
    # Tokenizing
    urls = []
    tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))
    tweet_text = re.sub(r"http\S+", "", tweet_text)
    # tokens = nltk.TweetTokenizer().tokenize(re.sub(r'([\d,.]+)','', tweet_text.lower()))

    # Setting url value (whether the tweet contains http link) and filter http links
    url=0
    for token in tokens:
        if token.startswith('http'):
            url=1
    tokens = [token for token in tokens if not token.startswith('http')]

    ## Stemming
    # porter = PorterStemmer()
    # tokens = [porter.stem(token) for token in tokens]

    # Filtering Stop words
    # from nltk.corpus import stopwords
    # stop_words = set(stopwords.words('english'))
    # tokens = [token for token in tokens if not token in stop_words]

    return tokens, url

def extract_urls(entities_dicts):
    if len(entities_dicts) < 1:
        return 0,[],[]

    urls = []
    urls_expanded = []

    key = 'url'
    key2 = 'expanded_url'
    # print(len(entities_dict))
    for i in entities_dicts:
        urls.append(i[key])
        urls_expanded.append(i[key2])
    return 1, urls, urls_expanded

def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    tweets_list = []
    total_tokens_l = []

    # Iterate through each tweet
    for tweet_obj in tweets:
        output_f = dict()
        # print(tweet_obj['text'])
        output_f['text']= tweet_obj['text']
        
        urls_dicts = tweet_obj['entities']['urls']

        output_f['verified'] = tweet_obj['user']['verified']

        tweets_list.append(output_f)

    unk_tokens_l = list(set(total_tokens_l))
    # print("Number of total tokens appeared: {}\nNumber of unique tokens appeared: {}\n".format(len(total_tokens_l), len(unk_tokens_l))) # number of tokens and unique tokens

    return tweets_list

In [37]:
data_path = "../pheme-rnr-dataset"
loader = data_loader(data_path)
jsonFiles_data = loader.getdata()
# dev_path = "./Other's code/rumoureval-master/data/dev"
# jsonFiles_dev = data_loader().getdata(dev_path)

The number of json files are: 5802


In [38]:
# len(jsonFiles_data[0][0])
# jsonFiles_data[0][0]

In [39]:
data_lists, y_lists = loader.extract_data(jsonFiles_data)
print("(Data of Root tweets) X: {} y: {}".format(len(data_lists),len(y_lists)))

(Data of Root tweets) X: 1 y: 1


In [41]:
data = pd.DataFrame(flatten_tweets(data_lists[0]))
y = pd.DataFrame(y_lists[0])

In [42]:
concat = pd.concat([data,y], axis=1)

In [43]:
concat.value_counts('isRumor')

isRumor
0    3830
1    1972
dtype: int64

In [44]:
concat.to_csv('./data/_PHEME_text.csv', index = False)

In [45]:
# d = pd.read_csv("./data/_PHEME_text.csv")

---

In [90]:
path = "../pheme-rnr-dataset"
events = ['charliehebdo', 'ferguson',
          'germanwings-crash', 'ottawashooting', 'sydneysiege']
jsons = []

In [91]:
for i, event in enumerate(events):
    # print(i, event)
    jsons.append(glob('%s/%s/*/*/source-tweet/*.json' % (path, event)))


In [100]:
len(jsons)

5

In [99]:
targets = []
features = []
for index, dataset in enumerate(jsons):
    targetEvent = []
    dataEvent = []
    count = 0  # help var
    for jsonFile in dataset:
        count += 1
        if jsonFile.find("non-rumours") == -1:
            targetEvent.append(1)
        else:
            targetEvent.append(0)

        with open(jsonFile, 'r') as f:
            for l in f.readlines():
                if not l.strip():  # skip empty lines
                    continue
                json_data = json.loads(l)
                # print (json_data,"\n\n")
                dataEvent.append(json_data)
    print(index, events[index], len(targetEvent), len(dataEvent))
    targets.append(targetEvent)
    features.append(dataEvent)


0 charliehebdo 2079 2079
1 ferguson 1143 1143
2 germanwings-crash 469 469
3 ottawashooting 890 890
4 sydneysiege 1221 1221


In [129]:
extracted_features = []

extracted = []
# total_tokens_l = []

# Iterate through each tweet
for obj_list in features:
    extracted_event = []
    for obj in obj_list:
        output_f = dict()
        # print(tweet_obj['text'])
        output_f['text'] = obj['text']
        # urls_dicts = obj['entities']['urls']
        # output_f['verified'] = obj['user']['verified']
        extracted_event.append(output_f)
    extracted_features.append(extracted_event)
# unk_tokens_l = list(set(total_tokens_l))
# print("Number of total tokens appeared: {}\nNumber of unique tokens appeared: {}\n".format(
# len(total_tokens_l), len(unk_tokens_l)))  # number of tokens and unique tokens


In [156]:
extracted_df = []
for i, data in enumerate(extracted_features):
    temp = pd.DataFrame(data)
    temp["Event"] = events[i]
    extracted_df.append(pd.DataFrame(temp))


In [157]:
final = pd.concat(extracted_df, ignore_index=True)


In [158]:
final

,text,Event
0,BREAKING: Armed man takes hostage in kosher gr...,charliehebdo
1,"#CharlieHebdo killers dead, confirmed by genda...",charliehebdo
2,"Top French cartoonists Charb, Cabu, Wolinski, ...",charliehebdo
3,Police have surrounded the area where the #Cha...,charliehebdo
4,PHOTO: Armed gunmen face police officers near ...,charliehebdo
...,...,...
5797,'I'll ride with you' http://t.co/llZnuCAzg5 Au...,sydneysiege
5798,Canada's thoughts and prayers are with our Aus...,sydneysiege
5799,Every non-muslim in the world must watch this ...,sydneysiege
5800,Suspect in Sydney cafe siege identified as Man...,sydneysiege
